### NOTE: you need to set your SOLANARPC_HTTP_URI env var

In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [2]:
from external_klend_utils import *
import time
import os

# Fetch Current Loan Data from On-Chain

In [12]:
async def get_all_market_data(market):
    start_time = time.time()
    
    uri = os.getenv("SOLANARPC_HTTP_URI")

    # make kamino_lend_program
    kamino_lend_program = get_kamino_lend_program(uri)

    reserves_configs = await get_reserves_configs(kamino_lend_program, market)
    elevation_group_dict = await get_elevation_group_dict(market)
    loan_metrics_df = await get_loans_metrics(reserves_configs, elevation_group_dict, lending_market=market)

    time_taken = time.time() - start_time
    logging.info(f"time_taken = {time_taken:.2f}s")
    
    print(f"Number of reserves = {len(reserves_configs)}")
    print(f"Number of elevation groups = {len(elevation_group_dict)}")
    print(f"Number of loans = {loan_metrics_df.shape[0]:,}")
    return reserves_configs, elevation_group_dict,loan_metrics_df

In [13]:
lending_market = LENDING_MARKETS['altcoin_market']
reserves_configs, elevation_group_dict,loan_metrics_df = await get_all_market_data(lending_market)

Number of reserves = 14
Number of elevation groups = 0
Number of loans = 22,093


In [5]:
mint_to_str_map, str_to_mint_map = await get_scope_mints_to_str_map()

In [6]:
# token reserve tickers in this market
[mint_to_str_map[k] for k in reserves_configs.keys()]

['DRIFT',
 'KMNO',
 'PRCL',
 'USDH',
 'INF',
 'TNSR',
 'USDC',
 'BONK',
 'W',
 'JTO',
 'WEN',
 'PYTH',
 'WIF',
 'JUP']

In [7]:
# token reserve tickers in this market which are shown in UI (status == 0)
reserves_configs_keys_ui = {
    k: r
    for k, r in reserves_configs.items()
    if r["account"]["config"]["status"] == 0
}

[mint_to_str_map[k] for k in reserves_configs_keys_ui.keys()]

['USDH',
 'INF',
 'TNSR',
 'USDC',
 'BONK',
 'W',
 'JTO',
 'WEN',
 'PYTH',
 'WIF',
 'JUP']

In [8]:
# view key cols
loan_metrics_df_copy = loan_metrics_df.copy()
cols = [
    "owner",
    "elevation_group",
    "current_ltv",
    "max_ltv",
    "unhealthy_ltv",
    "dist_to_liq",
    "total_deposit_usd",
    "total_borrow_usd",
    "net_value",
]
loan_metrics_df_copy = loan_metrics_df_copy[cols]
loan_metrics_df_copy = loan_metrics_df_copy[loan_metrics_df_copy.total_borrow_usd > 0.0]
loan_metrics_df_copy.sort_values(by='dist_to_liq', ascending=True, inplace=True)
loan_metrics_df_copy.head(5)

,owner,elevation_group,current_ltv,max_ltv,unhealthy_ltv,dist_to_liq,total_deposit_usd,total_borrow_usd,net_value
public_key,,,,,,,,,
4aALVBGvh2czH4rLteMY3CFaHf461bMcv1pmiFBHvqVE,CbQGxnGEdfBK6BsPBB3so3PVRuRFx2oDaDupGfuYhBvA,0,0.148684,0.10,0.20,0.051316,1.992942e+01,0.592638,1.933678e+01
EjcKFuHJtq3CraQjvo3CXNcwf2PtNZKxaeUiRQMVYhNg,Hrix7ZDGtBFsVLUAGQiXSYT3aTb3CBLLfAjtgfHtCqiF,0,0.678291,0.65,0.75,0.071709,2.018593e-01,0.136919,6.494001e-02
8rRg3ftQjfN3vFzERQwvZ3Rnv4sgY7jva4ZhMYPMHr2j,337tAvLe7hgoeu2FC3yLHpnqyr2RajhnDVtjEMes9YfJ,0,0.127842,0.10,0.20,0.072158,8.378420e+04,10711.114674,7.307308e+04
2xJdmVFRecZ9FhDzHs3GKHDY5qT6tNeswfDcgCb34Ghq,H7mtTUY1KNcAM9ZPUL9hYsB3z4z2g67QdtX3wdsASc4v,0,0.127270,0.10,0.20,0.072730,1.339238e+06,170445.482716,1.168793e+06
Dric7ceWrTktdxoosvwXJBRTUnAxVgt489RvKB76txkU,A1HcDsVATuWg9JHuJXsc6oi66hBnLHyFS8YmDrAAdnv4,0,0.126935,0.10,0.20,0.073065,7.050872e+04,8941.547379,6.150033e+04


In [9]:
# get IR curves
interest_rates = {}
for token in reserves_configs.keys():
    (
        df_interpolated,
        borrow_curve_df,
        current_util,
        current_borrow_rate,
        current_borrow_rate_apy,
        current_supply_rate,
        current_supply_rate_apy,
    ) = get_ir_curve(reserves_configs, token)
    interest_rates[token] = {
        "df_interpolated": df_interpolated,
        "borrow_curve_df": borrow_curve_df,
        "current_util": current_util,
        "current_borrow_rate": current_borrow_rate,
        "current_borrow_rate_apy": current_borrow_rate_apy,
        "current_supply_rate": current_supply_rate,
        "current_supply_rate_apy": current_supply_rate_apy,
    }
interest_rates.keys()

dict_keys(['DriFtupJYLTosbwoN8koMbEYSx54aFAVLddWsbksjwg7', 'KMNo3nJsBXfcpJTVhZcXLW7RmTwTt4GVFE7suUBo9sS', '4LLbsb5ReP3yEtYzmXewyGjcir5uXtKFURtaEUVC2AHs', 'USDH1SM1ojwWUga67PGrgFWUHibbjqMvuMaDkRJTgkX', '5oVNBeEEQvYi1cX3ir8Dx5n1P7pdxydbGF2X4TxVusJm', 'TNSRxcUxoT9xBG3de7PiJyTDYu7kskLqcpddxnEJAS6', 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v', 'DezXAZ8z7PnrnRJjz3wXBoRgixCa6xjnB7YaB1pPB263', '85VBFQZC9TZkfaptBWjvUw7YbZjy52A6mjtPGjstQAmQ', 'jtojtomepa8beP8AuQc6eXt5FriJwfFMwQx2v2f9mCL', 'WENWENvqqNya429ubCdR81ZmD69brwQaaBYY6p3LCpk', 'HZ1JovNiVvGrGNiiYvEozEVgZ58xaU3RKwX8eACQBCt3', 'EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm', 'JUPyiwrYJFskUPiHa7hkeR8VUtAeFoSYbKedZNsDvCN'])

In [10]:
interest_rates['EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v']['borrow_curve_df']

,utilization_rate,borrow_rate,supply_rate
0,0.0,0.0,0.0
1,85.0,50.0,34.0
2,90.0,75.0,54.0
3,92.0,100.0,73.6
4,95.0,150.0,114.0
5,100.0,250.0,200.0
